## Day3: Agenda

- Regular Expressions
- JSON
- Date Time
- Hive
    - Architecture
    - ORC/Parquet/AVRO
    - optimizations

In [7]:
sentence = '''My house number is E 1201, on 1st street.
My phone: 9999988888
My mail: abc@xyz.com'''

In [3]:
'1st'.isdigit()

False

In [5]:
sentence.split()

['My',
 'house',
 'number',
 'is',
 'E',
 '1201,',
 'on',
 '1st',
 'street.',
 'My',
 'phone:',
 '9999988888']

In [4]:
for word in sentence.split():
    if word.isdigit():
        print(word)

9999988888


In [8]:
import re

In [10]:
numpattern = '\d+'    # digit
re.findall(numpattern, sentence)

['1201', '1', '9999988888']

In [28]:
#         user @ domain [.] extension
mailpattern = r'(\w+)@(\w+)\.\w+'       # [A-Za-z0-9_]
re.findall(mailpattern, sentence)

[('abc', 'xyz')]

In [13]:
spark

In [15]:
mpg = spark.read.csv('mpg.csv', header=True)
mpg

DataFrame[mpg: string, cylinders: string, displacement: string, horsepower: string, weight: string, acceleration: string, model_year: string, origin: string, name: string]

In [16]:
mpg.show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|
+----+---------+------------+----------+------+------------+----------+------+--------------------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|   usa|    ford galaxie 500|
|14.0|        8|       454.0|     220.0|  4354|         9.0|        70|   usa|    chevrolet impala|


In [19]:
mpg.groupby('origin').count().show()

+------+-----+
|origin|count|
+------+-----+
| japan|   79|
|europe|   70|
|   usa|  249|
+------+-----+



In [20]:
mpg.select('name').show()

+--------------------+
|                name|
+--------------------+
|chevrolet chevell...|
|   buick skylark 320|
|  plymouth satellite|
|       amc rebel sst|
|         ford torino|
|    ford galaxie 500|
|    chevrolet impala|
|   plymouth fury iii|
|    pontiac catalina|
|  amc ambassador dpl|
| dodge challenger se|
|  plymouth 'cuda 340|
|chevrolet monte c...|
|buick estate wago...|
|toyota corona mar...|
|     plymouth duster|
|          amc hornet|
|       ford maverick|
|        datsun pl510|
|volkswagen 1131 d...|
+--------------------+
only showing top 20 rows



In [21]:
import pyspark.sql.functions as F

In [29]:
#                                   column  pattern+group  group-id
maker = mpg.select(F.regexp_extract('name', r'(\w+) (.*)', 1).alias('maker'))
maker.show()

+----------+
|     maker|
+----------+
| chevrolet|
|     buick|
|  plymouth|
|       amc|
|      ford|
|      ford|
| chevrolet|
|  plymouth|
|   pontiac|
|       amc|
|     dodge|
|  plymouth|
| chevrolet|
|     buick|
|    toyota|
|  plymouth|
|       amc|
|      ford|
|    datsun|
|volkswagen|
+----------+
only showing top 20 rows



In [25]:
maker.groupby('maker').count().show()

+---------+-----+
|    maker|count|
+---------+-----+
|    buick|   17|
|  pontiac|   16|
| mercedes|    3|
|   toyota|   25|
|     saab|    4|
|      amc|   28|
|       vw|    6|
|  peugeot|    8|
| chrysler|    6|
| plymouth|   31|
|vokswagen|    1|
|    chevy|    3|
|     audi|    7|
|   datsun|   23|
|      bmw|    2|
|    dodge|   28|
|     ford|   51|
|  toyouta|    1|
|    capri|    1|
| cadillac|    2|
+---------+-----+
only showing top 20 rows



In [30]:
withmaker = mpg.withColumn('maker', F.regexp_extract('name', r'(\w+) (.*)', 1))
withmaker.show()

+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
| mpg|cylinders|displacement|horsepower|weight|acceleration|model_year|origin|                name|     maker|
+----+---------+------------+----------+------+------------+----------+------+--------------------+----------+
|18.0|        8|       307.0|     130.0|  3504|        12.0|        70|   usa|chevrolet chevell...| chevrolet|
|15.0|        8|       350.0|     165.0|  3693|        11.5|        70|   usa|   buick skylark 320|     buick|
|18.0|        8|       318.0|     150.0|  3436|        11.0|        70|   usa|  plymouth satellite|  plymouth|
|16.0|        8|       304.0|     150.0|  3433|        12.0|        70|   usa|       amc rebel sst|       amc|
|17.0|        8|       302.0|     140.0|  3449|        10.5|        70|   usa|         ford torino|      ford|
|15.0|        8|       429.0|     198.0|  4341|        10.0|        70|   usa|    ford galaxie 500|      ford|
|

In [31]:
withmaker.select(F.concat_ws(' ', 'maker', 'origin')).show()

+---------------------------+
|concat_ws( , maker, origin)|
+---------------------------+
|              chevrolet usa|
|                  buick usa|
|               plymouth usa|
|                    amc usa|
|                   ford usa|
|                   ford usa|
|              chevrolet usa|
|               plymouth usa|
|                pontiac usa|
|                    amc usa|
|                  dodge usa|
|               plymouth usa|
|              chevrolet usa|
|                  buick usa|
|               toyota japan|
|               plymouth usa|
|                    amc usa|
|                   ford usa|
|               datsun japan|
|          volkswagen europe|
+---------------------------+
only showing top 20 rows



## Date time operations

In [32]:
ss = spark.read.csv('superstore.csv', header=True, inferSchema=True)
ss.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+
|     1|CA-2017-152156|08/11/2017|11/11/2017|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|  261.96|
|     2|CA-2017-152156|0

In [34]:
ss

DataFrame[Row ID: int, Order ID: string, Order Date: string, Ship Date: string, Ship Mode: string, Customer ID: string, Customer Name: string, Segment: string, Country: string, City: string, State: string, Postal Code: int, Region: string, Product ID: string, Category: string, Sub-Category: string, Product Name: string, Sales: string]

In [36]:
ss = ss.withColumn('odate', F.to_date('Order Date', 'dd/MM/yyyy'))
ss = ss.withColumn('sdate', F.to_date('Ship Date', 'dd/MM/yyyy'))
ss

DataFrame[Row ID: int, Order ID: string, Order Date: string, Ship Date: string, Ship Mode: string, Customer ID: string, Customer Name: string, Segment: string, Country: string, City: string, State: string, Postal Code: int, Region: string, Product ID: string, Category: string, Sub-Category: string, Product Name: string, Sales: string, odate: date, sdate: date]

In [37]:
ss.select(F.dayofweek('odate')).show()

+----------------+
|dayofweek(odate)|
+----------------+
|               4|
|               4|
|               2|
|               3|
|               3|
|               3|
|               3|
|               3|
|               3|
|               3|
|               3|
|               3|
|               1|
|               3|
|               3|
|               3|
|               4|
|               4|
|               5|
|               5|
+----------------+
only showing top 20 rows



In [41]:
ss.select(F.datediff('sdate', 'odate').alias('shipdelay')).groupby('shipdelay').count().show()

+---------+-----+
|shipdelay|count|
+---------+-----+
|        1|  363|
|        6| 1170|
|        3|  978|
|        5| 2147|
|        4| 2718|
|        7|  615|
|        2| 1295|
|        0|  514|
+---------+-----+



In [42]:
# days since shipped
ss.select(F.datediff(F.current_date(), 'sdate')).show()

+-------------------------------+
|datediff(current_date(), sdate)|
+-------------------------------+
|                           2746|
|                           2746|
|                           2894|
|                           3135|
|                           3135|
|                           3627|
|                           3627|
|                           3627|
|                           3627|
|                           3627|
|                           3627|
|                           3627|
|                           2586|
|                           2717|
|                           3096|
|                           3096|
|                           3470|
|                           3657|
|                           3548|
|                           3548|
+-------------------------------+
only showing top 20 rows



In [44]:
ss.select(F.months_between(F.current_date(), 'sdate')).show()

+-------------------------------------------+
|months_between(current_date(), sdate, true)|
+-------------------------------------------+
|                                90.25806452|
|                                90.25806452|
|                                95.09677419|
|                               103.03225806|
|                               103.03225806|
|                               119.16129032|
|                               119.16129032|
|                               119.16129032|
|                               119.16129032|
|                               119.16129032|
|                               119.16129032|
|                               119.16129032|
|                                84.96774194|
|                                89.29032258|
|                               101.77419355|
|                               101.77419355|
|                               114.03225806|
|                               120.12903226|
|                               11

25/05/19 14:47:05 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1042069 ms exceeds timeout 120000 ms
25/05/19 14:47:05 WARN SparkContext: Killing executors is not supported by current scheduler.


In [ ]:
F.